Merge all the STAR counts from GDC - Matched normal samples 

In [ ]:
import os
import pandas as pd
dataframes_list = []
# Get the list of all directories in the current directory
directories = [d for d in os.listdir('.') if os.path.isdir(d)]

# Loop over each directory
for directory in directories:
    dir_path = os.path.join('.', directory)  # Get the full path of the directory
    subdirectories = [d for d in os.listdir(dir_path) if os.path.isdir(os.path.join(dir_path, d))]

    # Loop over each subdirectory
    for sub_dir in subdirectories:
        sub_dir_path = os.path.join(dir_path, sub_dir)  # Get the full path of the subdirectory

        # Get the list of all files in the current subdirectory
        files = os.listdir(sub_dir_path)

        # Filter for .tsv files and process them
        tsv_files = [item for item in files if item.endswith('augmented_star_gene_counts.tsv')][0]
        
        file_path = os.path.join(sub_dir_path, tsv_files)  # Get the full path of the .tsv file
        # Read the .tsv file with pandas, selecting the second and fourth columns
        
        df = pd.read_csv(file_path, sep='\t', skiprows=1, usecols=['gene_id', 'unstranded'])
        
        # Rename the columns
        df.columns = ['gene_id', tsv_files]
        df.dropna(inplace=True)
        df.set_index('gene_id', inplace=True, drop=True)
        dataframes_list.append(df)
        
merged_df = pd.concat(dataframes_list, join='inner',axis=1)

merged_df.to_csv('TCGA_normal.csv')

In [ ]:
df = pd.read_csv('TCGA_normal.csv')

Drop the first few row 

In [ ]:
df.drop([0,1,2,3],inplace=True)

In [ ]:
df.to_csv('TCGA_normal.csv', index=False)

Merge all the sample sheet

In [ ]:
import os
import pandas as pd

# Directory where the files are stored (assuming current directory for this example)
directory = '.'

# Initialize an empty list to store file paths
sample_sheet_files = []

# Loop through the files in the directory
for filename in os.listdir(directory):
    if "gdc_sample_sheet" in filename and filename.endswith(".tsv"):
        # Add the full path of the file to the list
        file_path = os.path.join(directory, filename)
        sample_sheet_files.append(file_path)

# Concatenate all the files into a single dataframe
df_concatenated = pd.concat((pd.read_csv(file, sep='\t') for file in sample_sheet_files), ignore_index=True)

# Display the first few rows of the concatenated dataframe
df_concatenated.head()

Create a dictionary to map the filename (current column name of the 'TCGA_normal.csv') to sample name 

In [ ]:
mappingDict = {}
for ind, row in df_concatenated.iterrows():
    if row[1] not in mappingDict:
        mappingDict[row[1]] = row[6]
    else:
        print(row)
    

Perform mapping

In [ ]:
colnamelst = ['gene_id']
for i in df.columns[1:]:
    colnamelst.append(mappingDict[i])

Replace column name 

In [ ]:
df.columns = colnamelst

In [ ]:
df.to_csv('TCGA_normal_.csv', index=False)